## Building a Regression Model using Keras

### Table of contents
1. Donwnload and clean dataset
2. Import Keras
3. Build a Neural Network
4. Train and Test the Neural Network

### Download and Clean Dataset

We will start by importing the necessary libraries for the project

In [1]:
import time
import os
import pandas as pd
import numpy as np


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
conda install -c conda-forge keras

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


The dataset that we will use is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used. The ingredients include:
1. Cement
2. Blast furnace slag
3. Fly ash
4. Water
5. Superplasticizer
6. Coarse aggregate
7. Fine aggregate

We will download the data and read it into a pandas daataframe

In [3]:
concrete_df = pd.read_csv("/Users/juan/Downloads/concrete_data.csv")
concrete_df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The first sample consists of 540 cubic meters of cement, 0 cubic meters of blast furnace slag and fly ash, 162 cubic meters of water, 2.5 cubic meters of superplasticizer, 1040 cubic meters of fine aggregate, 676 cubic meters of fine aggregate and has a compressive strength of 79.99 MPa at 28 days.

Check how many data points we have in the data frame

In [4]:
concrete_df.shape

(1030, 9)

The next step is to check the dataset for any missing values

In [5]:
concrete_df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The dataset looks clean so we can proceed to build our model

#### Split data intro predictors and target

In [7]:
concrete_df_columns = concrete_df.columns
predictors = concrete_df[concrete_df_columns[concrete_df_columns != "Strength"]] # All columns except Strength
target = concrete_df["Strength"] #Only the column Strength

Check the first rows for the predictors and target data

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to n_cols variables, because we will need it when we build our network

In [10]:
n_cols = predictors.shape[1] #number of predictors

### Importing packages from Keras library

Let's import the packages from the Keras library that we will need to build our regression model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Part A: Build a baseline model

Use the Keras library to build a neural network with the following:

One hidden layer of 10 nodes, and a ReLU activation function

Use the adam optimizer and the mean squared error as the loss function.

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

Train the model on the training data using 50 epochs.

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

Report the mean and the standard deviation of the mean squared errors.

#### Build the neural network model

Let's create a function that defines our regression model and we can call it to create our model

In [12]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_cols,)))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

####  Train and test the model

We will create the regression model

In [13]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


Randomly split the data into a training and test sets by holding 30%  of the data for testing. You can use the train_test_split helper function 

In [14]:
X = predictors
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)


Train the model on the training data using 50 epochs

In [15]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 0s - loss: 180527.1437
Epoch 2/50
 - 0s - loss: 73816.3552
Epoch 3/50
 - 0s - loss: 24301.4264
Epoch 4/50
 - 0s - loss: 9429.0605
Epoch 5/50
 - 0s - loss: 6735.8674
Epoch 6/50
 - 0s - loss: 6215.4641
Epoch 7/50
 - 0s - loss: 5925.8628
Epoch 8/50
 - 0s - loss: 5623.9222
Epoch 9/50
 - 0s - loss: 5337.4198
Epoch 10/50
 - 0s - loss: 5047.6248
Epoch 11/50
 - 0s - loss: 4774.2336
Epoch 12/50
 - 0s - loss: 4507.4581
Epoch 13/50
 - 0s - loss: 4256.2851
Epoch 14/50
 - 0s - loss: 4008.5407
Epoch 15/50
 - 0s - loss: 3782.1135
Epoch 16/50
 - 0s - loss: 3562.5313
Epoch 17/50
 - 0s - loss: 3354.1346
Epoch 18/50
 - 0s - loss: 3162.2124
Epoch 19/50
 - 0s - loss: 2972.0130
Epoch 20/50
 - 0s - loss: 2795.3107
Epoch 21/50
 - 0s - loss: 2636.3989
Epoch 22/50
 - 0s - loss: 2478.9551
Epoch 23/50
 - 0s - loss: 2335.0443
Epoch 24/50
 - 0s - loss: 2198.9506
Epoch 25/50
 - 0s - loss: 2076.0854
Epoch 26/50
 - 0s - loss: 1960.8697
Epoch 27/50
 - 0s - l

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.    
    

In [16]:
y_hat = model.predict(X_test)   
mse = mean_squared_error(y_test, y_hat)

In [17]:
print(mse)

605.8291324037887


Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors

We will create one function that performs steps 1 to 3 and another function that iterates 50 times and creates the list of 50 mean squared errors

In [18]:
def get_mean_squared_error(compiled_model, X, y, epochs=50, verbose=1):
    """Get report (dataframe) of two metrics: 
    The mean and the standard deviation of the mean squared errors
    """   
    
    # 1. Randomly split the data into a training and test sets by holding 30% 
    # of the data for testing. You can use the train_test_split helper function 
    # from Scikit-learn. 
    X = predictors
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
    print("Training set: ", X_train.shape, y_train.shape)
    print("Testing set: ", X_test.shape, y_test.shape)
    
    
    # 2. Train the model on the training data using 50 epochs.
    # Fit the built model with training set
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)    

    # 3. Evaluate the model on the test data and compute the mean squared error 
    # between the predicted concrete strength and the actual concrete strength. 
    # You can use the mean_squared_error function from Scikit-learn.    
    y_hat = model.predict(X_test)    
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [19]:
#Function to round the calculation of the mean and std deviation to 2 decimal places
def get_round(score, num_of_digits=2):
    """Get round with given number of decimal digits 
    """
    return round(score, num_of_digits)

#Function to calculate the mean of the list of mean squared errors
def get_mean(list_of_mse_scores):
    """Get mean
    """
    if list_of_mse_scores:
        return get_round(np.mean(list_of_mse_scores))
    return None

#Function to calculate the standard deviation of the list of mean squared errors
def get_standard_deviation(list_of_mse_scores):
    """Get standard deviation
    """
    if list_of_mse_scores:
        return get_round(np.std(list_of_mse_scores))
    return None




#Function to iterate and calculate the mean squared error
def get_mean_and_std_of_mse(df_X, 
                            df_y, 
                            compiled_model,                
                            max_iteration=50, 
                            epochs=50, 
                            verbose=0):
    """Generate the mean and the standard deviation of the mean squared errors 
    """
    # Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.    
    list_of_mean_squared_errors = []
    for i in range(max_iteration):
        start_time = time.time()
        print("-" * 36)
        print("Processing current number of iteration : {}".format(i+1))        
        mse = get_mean_squared_error(compiled_model, df_X, df_y, epochs=epochs, verbose=verbose)
        list_of_mean_squared_errors.append(mse)
        print("Duration (seconds): {}".format(time.time()-start_time))
    # end for

    print("Finished - {} times.\nAnd the list of mean squared errors : {}".format(max_iteration,
                                                                                  list_of_mean_squared_errors))
    
    mean_mse = get_mean(list_of_mean_squared_errors)
    std_mse = get_standard_deviation(list_of_mean_squared_errors)

    print("-" * 72)
    print("The mean and the standard deviation of the mean squared errors are: {} and {}, respectively".format(
           mean_mse, std_mse))
    
    return mean_mse, std_mse




In [20]:
max_iteration = 50
epochs = 50
verbose = 2

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Epoch 1/50
 - 0s - loss: 119351.8213
Epoch 2/50
 - 0s - loss: 44068.6167
Epoch 3/50
 - 0s - loss: 11587.9586
Epoch 4/50
 - 0s - loss: 3815.3857
Epoch 5/50
 - 0s - loss: 3047.8626
Epoch 6/50
 - 0s - loss: 2971.1660
Epoch 7/50
 - 0s - loss: 2878.7121
Epoch 8/50
 - 0s - loss: 2793.7753
Epoch 9/50
 - 0s - loss: 2701.1468
Epoch 10/50
 - 0s - loss: 2610.0612
Epoch 11/50
 - 0s - loss: 2522.3222
Epoch 12/50
 - 0s - loss: 2437.0822
Epoch 13/50
 - 0s - loss: 2351.9643
Epoch 14/50
 - 0s - loss: 2267.0414
Epoch 15/50
 - 0s - loss: 2188.4220
Epoch 16/50
 - 0s - loss: 2110.2339
Epoch 17/50
 - 0s - loss: 2034.0986
Epoch 18/50
 - 0s - loss: 1960.7441
Epoch 19/50
 - 0s - loss: 1890.8609
Epoch 20/50
 - 0s - loss: 1826.7647
Epoch 21/50
 - 0s - loss: 1757.2713
Epoch 22/50
 - 0s - loss: 1695.7342
Epoch 23/50
 - 0s - loss: 1638.9417
Epoch 24/50
 - 0s - loss: 1581.0371


### Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of not normalized data is:

In [21]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [22]:
name_of_case = "Baseline-not normalized (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline = get_report(name_of_case, mean_mse, std_mse)
df_baseline

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),82.33,102.66


## Part B: Normalized data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.
How does the mean of the mean squared errors compare to that from Step A?

### Data before normalization

In [23]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


### Data after normalization

We will normalize the data by substracting the mean and dividing by the standard deviation

In [24]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [25]:
n_colsnorm = predictors_norm.shape[1] #number of predictors

Defining the regression model as above

In [26]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_colsnorm,)))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Creating the regression model

In [27]:
model = regression_model()

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function

In [28]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=24)   
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)


Train the model on the training data using 50 epochs

In [29]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 0s - loss: 1623.5850
Epoch 2/50
 - 0s - loss: 1608.4249
Epoch 3/50
 - 0s - loss: 1593.4508
Epoch 4/50
 - 0s - loss: 1579.0658
Epoch 5/50
 - 0s - loss: 1564.2792
Epoch 6/50
 - 0s - loss: 1549.4693
Epoch 7/50
 - 0s - loss: 1534.1506
Epoch 8/50
 - 0s - loss: 1518.3228
Epoch 9/50
 - 0s - loss: 1501.7177
Epoch 10/50
 - 0s - loss: 1484.3600
Epoch 11/50
 - 0s - loss: 1466.2228
Epoch 12/50
 - 0s - loss: 1447.0411
Epoch 13/50
 - 0s - loss: 1426.6101
Epoch 14/50
 - 0s - loss: 1405.6983
Epoch 15/50
 - 0s - loss: 1383.4877
Epoch 16/50
 - 0s - loss: 1360.5316
Epoch 17/50
 - 0s - loss: 1336.9053
Epoch 18/50
 - 0s - loss: 1312.4215
Epoch 19/50
 - 0s - loss: 1287.4737
Epoch 20/50
 - 0s - loss: 1261.4789
Epoch 21/50
 - 0s - loss: 1235.5406
Epoch 22/50
 - 0s - loss: 1208.8780
Epoch 23/50
 - 0s - loss: 1181.5878
Epoch 24/50
 - 0s - loss: 1154.1139
Epoch 25/50
 - 0s - loss: 1126.2791
Epoch 26/50
 - 0s - loss: 1098.2967
Epoch 27/50
 - 0s - loss: 1070.0182
Epoch 28/50
 - 0s - loss: 1041.7842
E

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [30]:
y_hat = model.predict(X_test)   
mse = mean_squared_error(y_test, y_hat)

In [31]:
print(mse)

430.8369270906882


For the case of not normalized data that we ran in Part A mse = 425.59, for this case with normalized data mse=281.52

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors

In [32]:
def get_mean_squared_error(compiled_model, X, y, epochs=50, verbose=1):
    """Get report (dataframe) of two metrics: 
    The mean and the standard deviation of the mean squared errors
    """   
    
    # 1. Randomly split the data into a training and test sets by holding 30% 
    # of the data for testing. You can use the train_test_split helper function 
    # from Scikit-learn. 
    X = predictors_norm
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
    print("Training set: ", X_train.shape, y_train.shape)
    print("Testing set: ", X_test.shape, y_test.shape)
    
    
    # 2. Train the model on the training data using 50 epochs.
    # Fit the built model with training set
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)    

    # 3. Evaluate the model on the test data and compute the mean squared error 
    # between the predicted concrete strength and the actual concrete strength. 
    # You can use the mean_squared_error function from Scikit-learn.    
    y_hat = model.predict(X_test)    
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [33]:
#Function to round the calculation of the mean and std deviation to 2 decimal places
def get_round(score, num_of_digits=2):
    """Get round with given number of decimal digits 
    """
    return round(score, num_of_digits)

#Function to calculate the mean of the list of mean squared errors
def get_mean(list_of_mse_scores):
    """Get mean
    """
    if list_of_mse_scores:
        return get_round(np.mean(list_of_mse_scores))
    return None

#Function to calculate the standard deviation of the list of mean squared errors
def get_standard_deviation(list_of_mse_scores):
    """Get standard deviation
    """
    if list_of_mse_scores:
        return get_round(np.std(list_of_mse_scores))
    return None




#Function to iterate and calculate the mean squared error
def get_mean_and_std_of_mse(df_X, 
                            df_y, 
                            compiled_model,                
                            max_iteration=50, 
                            epochs=50, 
                            verbose=0):
    """Generate the mean and the standard deviation of the mean squared errors 
    """
    # Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.    
    list_of_mean_squared_errors = []
    for i in range(max_iteration):
        start_time = time.time()
        print("-" * 36)
        print("Processing current number of iteration : {}".format(i+1))        
        mse = get_mean_squared_error(compiled_model, df_X, df_y, epochs=epochs, verbose=verbose)
        list_of_mean_squared_errors.append(mse)
        print("Duration (seconds): {}".format(time.time()-start_time))
    # end for

    print("Finished - {} times.\nAnd the list of mean squared errors : {}".format(max_iteration,
                                                                                  list_of_mean_squared_errors))
    
    mean_mse = get_mean(list_of_mean_squared_errors)
    std_mse = get_standard_deviation(list_of_mean_squared_errors)

    print("-" * 72)
    print("The mean and the standard deviation of the mean squared errors are: {} and {}, respectively".format(
           mean_mse, std_mse))
    
    return mean_mse, std_mse




In [34]:
max_iteration = 50
epochs = 50
verbose = 2

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors_norm, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Epoch 1/50
 - 0s - loss: 1607.7804
Epoch 2/50
 - 0s - loss: 1588.5265
Epoch 3/50
 - 0s - loss: 1570.0454
Epoch 4/50
 - 0s - loss: 1551.8686
Epoch 5/50
 - 0s - loss: 1534.1607
Epoch 6/50
 - 0s - loss: 1516.8524
Epoch 7/50
 - 0s - loss: 1499.7577
Epoch 8/50
 - 0s - loss: 1482.3267
Epoch 9/50
 - 0s - loss: 1465.2797
Epoch 10/50
 - 0s - loss: 1447.5409
Epoch 11/50
 - 0s - loss: 1429.9501
Epoch 12/50
 - 0s - loss: 1411.9260
Epoch 13/50
 - 0s - loss: 1393.5068
Epoch 14/50
 - 0s - loss: 1374.7789
Epoch 15/50
 - 0s - loss: 1355.7026
Epoch 16/50
 - 0s - loss: 1336.2381
Epoch 17/50
 - 0s - loss: 1316.2639
Epoch 18/50
 - 0s - loss: 1295.7872
Epoch 19/50
 - 0s - loss: 1274.8553
Epoch 20/50
 - 0s - loss: 1253.7118
Epoch 21/50
 - 0s - loss: 1231.5125
Epoch 22/50
 - 0s - loss: 1208.9604
Epoch 23/50
 - 0s - loss: 1186.4615
Epoch 24/50
 - 0s - loss: 1162.2975
Epoc

###  Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of not normalized data is:

In [35]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [36]:
name_of_case = "Baseline  normalized (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline_norm = get_report(name_of_case, mean_mse, std_mse)
df_baseline_norm

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline normalized (50 epochs),49.4,51.66


Comparing the case of not normalized data with the case of normalized data:

In [37]:
# Create a data frame with the summary
df_summary = pd.concat([df_baseline, df_baseline_norm], axis=0)

# Review the result dataframe
df_summary.reset_index(drop=True)


,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),82.33,102.66
1,Baseline normalized (50 epochs),49.40,51.66


### Part C: Normalized data with 100 epochs

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

#### Building the model with normalized data and increasing to 100 epochs

In [38]:
max_iteration = 50
epochs = 100
verbose = 0

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors_norm, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 3.3392791748046875
------------------------------------
Processing current number of iteration : 2
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 2.5032520294189453
------------------------------------
Processing current number of iteration : 3
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 2.5578949451446533
------------------------------------
Processing current number of iteration : 4
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 3.26336407661438
------------------------------------
Processing current number of iteration : 5
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 2.420630693435669
------------------------------------
Processing current number of iteration : 6
Training set:  (721, 8)

### Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of  normalized data running 100 epochs is:

In [39]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [40]:
name_of_case = "Baseline  normalized (100 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline_norm100 = get_report(name_of_case, mean_mse, std_mse)
df_baseline_norm100

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline normalized (100 epochs),40.77,16.02


Comparing the normalized cases ran with 50 epochs and 100 epochs 

In [41]:
# Create a data frame with the summary
df_summary = pd.concat([df_baseline, df_baseline_norm, df_baseline_norm100], axis=0)

# Review the result dataframe
df_summary.reset_index(drop=True)



,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),82.33,102.66
1,Baseline normalized (50 epochs),49.40,51.66
2,Baseline normalized (100 epochs),40.77,16.02


We can see from the result, that both the Mean of the  MSE (mean squared error) and the Std Deviation  of the MSE reduced when the number of epochs was increased from 50 to 100

### Part D: Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:

Three hidden layers, each of 10 nodes and ReLU activation function.
How does the mean of the mean squared errors compare to that from Step B?

Define the neural network with three hidden layers

In [42]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_colsnorm,)))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Create the regression model

In [43]:
model = regression_model()

#### Buildling the normalized model with 3 hidden layers and running 50 epochs

In [44]:
max_iteration = 50
epochs = 50
verbose = 0

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors_norm, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 4.236215114593506
------------------------------------
Processing current number of iteration : 2
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 1.7553222179412842
------------------------------------
Processing current number of iteration : 3
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 1.7203178405761719
------------------------------------
Processing current number of iteration : 4
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 1.7975847721099854
------------------------------------
Processing current number of iteration : 5
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Duration (seconds): 2.246209144592285
------------------------------------
Processing current number of iteration : 6
Training set:  (721, 8

### Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of  normalized data with a model of 3 hidden layers is:

In [45]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [48]:
name_of_case = "Baseline  normalized 3 hidden layers (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline_norm3hid = get_report(name_of_case, mean_mse, std_mse)
df_baseline_norm3hid

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline normalized 3 hidden layers (50 epochs),41.66,14.25


Comparing the normalized cases with one and three hidden layers

In [49]:
# Create a data frame with the summary
df_summary = pd.concat([df_baseline, df_baseline_norm, df_baseline_norm100, df_baseline_norm3hid], axis=0)

# Review the result dataframe
df_summary.reset_index(drop=True)

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),82.33,102.66
1,Baseline normalized (50 epochs),49.40,51.66
2,Baseline normalized (100 epochs),40.77,16.02
3,Baseline normalized 3 hidden layers (50 epochs),41.66,14.25


We can observe that increasing the number of hidden layers from one to three, resulted in a decrease in mean MSE and a more significant decrease in the std deviation of the mse